In [1]:
# Hey, I'm able to edit this

import sys
from pathlib import Path

# Add analysis tools to path
NOTEBOOK_DIR = Path.cwd()
ANALYSIS_ROOT = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == "notebooks" else NOTEBOOK_DIR
if str(ANALYSIS_ROOT) not in sys.path:
    sys.path.insert(0, str(ANALYSIS_ROOT))

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tools import wandb_io, config, paths
from tools.aggregations import runs_history_to_frame

## Configuration

In [2]:
# Sweep config
SWEEP_ID = "w7jmdcvc"
ENTITY = "thomasevers9"
PROJECT = "tdmpc2-tdmpc2"

# Plot config
MAX_STEP = 100_000  # This sweep runs for 100k steps
METRIC_KEY = "eval/episode_reward"

# Task name mapping: sweep task name -> baseline CSV name
# Only include mappings where names differ
TASK_NAME_MAP = {
    "ball_in_cup-catch": "cup-catch",
    "finger-turn_easy": "finger-turn-easy",
    "finger-turn_hard": "finger-turn-hard",
    "cartpole-balance_sparse": "cartpole-balance-sparse",
    "cartpole-swingup_sparse": "cartpole-swingup-sparse",
}

# Baseline roots
BASELINE_ROOTS = {
    "TDMPC2": paths.BASELINE_TDMPC2,
    "DreamerV3": paths.BASELINE_DREAMERV3,
    "SAC": paths.BASELINE_SAC,
    "EZ2": paths.PROJECT_ROOT / "results" / "ez2_parsed",
    "SimbaV2": paths.PROJECT_ROOT / "results" / "simbav2_parsed",
    "BMPC": paths.PROJECT_ROOT / "results" / "BMPC_parsed" / "dmcontrol",
}

# Colors for methods
COLORS = {
    "Ours": "#1f77b4",      # Blue
    "TDMPC2": "#2ca02c",    # Green
    "DreamerV3": "#ff7f0e", # Orange
    "EZ2": "#d62728",       # Red
    "SimbaV2": "#9467bd",   # Purple
    "SAC": "#8c564b",       # Brown
    "BMPC": "#17becf",      # Cyan
}

## Load Our Results from W&B

In [3]:
# Fetch sweep runs from W&B (with caching)
# Set force_refresh=True to pull latest data (may fail due to brotli bug)
runs, manifest, source = wandb_io.fetch_sweep_runs(
    entity=ENTITY,
    project=PROJECT,
    sweep_id=SWEEP_ID,
    history_keys=[METRIC_KEY, "_step"],
    use_cache=True,
    force_refresh=False,  # Use cached data; set True to force re-fetch
)
print(f"Loaded {len(runs)} runs from {source}")
print(f"Fetched at: {manifest.get('fetched_at', 'N/A')}")

Loaded 18 runs from cache
Fetched at: 2026-01-06T13:38:53.546055+00:00


In [4]:
# Convert to DataFrame
ours_df = runs_history_to_frame(
    runs,
    metric_key=METRIC_KEY,
    step_keys=["_step"],
    config_to_columns={"task": "task", "seed": "seed"},
)

# Normalize task names to match baseline naming
ours_df["task"] = ours_df["task"].replace(TASK_NAME_MAP)

# Rename metric column to 'reward' for consistency
ours_df = ours_df.rename(columns={METRIC_KEY: "reward"})

print(f"Our results: {len(ours_df)} rows")
print(f"Tasks: {sorted(ours_df['task'].unique().tolist())}")
print(f"Seeds: {sorted(ours_df['seed'].unique().tolist())}")
print(f"Max step: {ours_df['step'].max()}")
ours_df.head()

Our results: 180 rows
Tasks: ['acrobot-swingup', 'cartpole-swingup-sparse', 'cheetah-run', 'finger-spin', 'finger-turn-hard', 'hopper-hop', 'pendulum-swingup', 'quadruped-walk', 'reacher-hard']
Seeds: [1, 2]
Max step: 100000


,task,seed,run_id,step,reward
0,reacher-hard,1,39pq2i7z,10000,390.399994
1,reacher-hard,1,39pq2i7z,20000,757.000000
2,reacher-hard,1,39pq2i7z,30000,781.799988
3,reacher-hard,1,39pq2i7z,40000,812.599976
4,reacher-hard,1,39pq2i7z,50000,789.000000


## Load Baseline Results

In [5]:
def load_baseline_for_task(task: str, method: str, root: Path) -> pd.DataFrame:
    """Load baseline CSV for a task.
    
    Args:
        task: Normalized task name (e.g., 'cup-catch').
        method: Baseline method name.
        root: Path to baseline CSV directory.
    
    Returns:
        DataFrame with columns [step, reward, seed, task, method], or empty if not found.
    """
    csv_path = root / f"{task}.csv"
    if csv_path.exists():
        df = pd.read_csv(csv_path)
        df["task"] = task
        df["method"] = method
        return df
    return pd.DataFrame()


def load_all_baselines(tasks: list, max_step: int) -> pd.DataFrame:
    """Load all baseline results for given tasks.
    
    Args:
        tasks: List of normalized task names.
        max_step: Maximum step to include.
    
    Returns:
        Combined DataFrame of all baselines.
    """
    frames = []
    missing = {method: [] for method in BASELINE_ROOTS}
    
    for method, root in BASELINE_ROOTS.items():
        if not root.exists():
            print(f"Warning: {method} root not found: {root}")
            continue
            
        for task in tasks:
            df = load_baseline_for_task(task, method, root)
            if df.empty:
                missing[method].append(task)
            else:
                df = df[df["step"] <= max_step]
                frames.append(df)
    
    # Report missing
    for method, tasks_missing in missing.items():
        if tasks_missing:
            print(f"{method}: missing {len(tasks_missing)} tasks: {tasks_missing[:5]}..." 
                  if len(tasks_missing) > 5 else f"{method}: missing {tasks_missing}")
    
    if not frames:
        return pd.DataFrame()
    return pd.concat(frames, ignore_index=True)

In [6]:
# Get unique tasks from our results
our_tasks = sorted(ours_df["task"].unique().tolist())
print(f"Our tasks ({len(our_tasks)}): {our_tasks}")

Our tasks (9): ['acrobot-swingup', 'cartpole-swingup-sparse', 'cheetah-run', 'finger-spin', 'finger-turn-hard', 'hopper-hop', 'pendulum-swingup', 'quadruped-walk', 'reacher-hard']


In [7]:
# Load baselines
baselines_df = load_all_baselines(our_tasks, MAX_STEP)
print(f"\nBaseline results: {len(baselines_df)} rows")

if not baselines_df.empty:
    coverage = baselines_df.groupby('method')['task'].nunique()
    print(f"\nTask coverage per method:")
    for method, count in coverage.items():
        print(f"  {method}: {count}/{len(our_tasks)} tasks")


Baseline results: 876 rows

Task coverage per method:
  BMPC: 9/9 tasks
  DreamerV3: 9/9 tasks
  EZ2: 9/9 tasks
  SAC: 9/9 tasks
  SimbaV2: 9/9 tasks
  TDMPC2: 9/9 tasks


## Combine Data

In [8]:
# Add method column to our results
ours_df["method"] = "Ours"

# Filter our results to max step
ours_filtered = ours_df[ours_df["step"] <= MAX_STEP].copy()

# Combine all data
all_data = pd.concat([ours_filtered, baselines_df], ignore_index=True)
print(f"Total rows: {len(all_data)}")
print(f"Methods: {sorted(all_data['method'].unique().tolist())}")

Total rows: 1056
Methods: ['BMPC', 'DreamerV3', 'EZ2', 'Ours', 'SAC', 'SimbaV2', 'TDMPC2']


## Plotting Functions

In [9]:
def hex_to_rgba(hex_color: str, alpha: float) -> str:
    """Convert hex color to rgba string."""
    hex_color = hex_color.lstrip('#')
    r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return f"rgba({r},{g},{b},{alpha})"


def compute_stats(df: pd.DataFrame, group_cols: list) -> pd.DataFrame:
    """Compute mean and std across seeds."""
    return df.groupby(group_cols).agg(
        mean_reward=("reward", "mean"),
        std_reward=("reward", "std"),
        n_seeds=("reward", "count")
    ).reset_index()

In [10]:
def plot_task_comparison(
    data: pd.DataFrame,
    task: str,
    colors: dict,
) -> go.Figure:
    """Create a comparison plot for a single task.
    
    Args:
        data: DataFrame with columns [step, reward, method, task].
        task: Task name to plot.
        colors: Dict mapping method names to hex colors.
    
    Returns:
        Plotly Figure, or None if no data for task.
    """
    task_data = data[data["task"] == task]
    if task_data.empty:
        return None
    
    stats = compute_stats(task_data, ["method", "step"])
    fig = go.Figure()
    
    # Plot Ours first, then baselines
    methods = ["Ours"] + [m for m in sorted(stats["method"].unique()) if m != "Ours"]
    
    for method in methods:
        method_stats = stats[stats["method"] == method].sort_values("step")
        if method_stats.empty:
            continue
        
        color = colors.get(method, "#666666")
        x = method_stats["step"]
        y_mean = method_stats["mean_reward"]
        y_std = method_stats["std_reward"].fillna(0)
        
        # Shaded std region
        fig.add_trace(go.Scatter(
            x=pd.concat([x, x[::-1]]),
            y=pd.concat([y_mean + y_std, (y_mean - y_std)[::-1]]),
            fill="toself",
            fillcolor=hex_to_rgba(color, 0.2),
            line=dict(color="rgba(0,0,0,0)"),
            showlegend=False,
            hoverinfo="skip",
        ))
        
        # Mean line (solid for Ours, dashed for baselines)
        line_style = dict(color=color, width=2)
        if method != "Ours":
            line_style["dash"] = "dash"
        
        fig.add_trace(go.Scatter(
            x=x,
            y=y_mean,
            mode="lines",
            name=method,
            line=line_style,
            hovertemplate=f"{method}<br>Step: %{{x:,}}<br>Reward: %{{y:.1f}}<extra></extra>",
        ))
    
    fig.update_layout(
        title=dict(text=f"<b>{task}</b>", x=0.5),
        xaxis_title="Environment Steps",
        yaxis_title="Episode Reward",
        legend=dict(x=0.02, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
        hovermode="x unified",
        template="plotly_white",
        width=700,
        height=450,
    )
    
    return fig

## Individual Task Plots

In [11]:
# Generate plots for all tasks
figures = {}
for task in our_tasks:
    fig = plot_task_comparison(all_data, task, COLORS)
    if fig is not None:
        figures[task] = fig

print(f"Generated {len(figures)} task plots")

Generated 9 task plots


In [12]:
# Show first 4 plots
for task in list(figures.keys())[:4]:
    print(f"\n--- {task} ---")
    figures[task].show()


--- acrobot-swingup ---



--- cartpole-swingup-sparse ---



--- cheetah-run ---



--- finger-spin ---


## Grid View (All Tasks)

In [13]:
def create_grid_figure(
    data: pd.DataFrame,
    tasks: list,
    colors: dict,
    cols: int = 3,
) -> go.Figure:
    """Create grid of comparison plots for all tasks.
    
    Args:
        data: Combined DataFrame with all methods.
        tasks: List of task names to plot.
        colors: Dict mapping method names to hex colors.
        cols: Number of columns in grid.
    
    Returns:
        Plotly Figure with subplots.
    """
    n_tasks = len(tasks)
    rows = (n_tasks + cols - 1) // cols
    
    fig = make_subplots(
        rows=rows,
        cols=cols,
        subplot_titles=tasks,
        vertical_spacing=0.08,
        horizontal_spacing=0.05,
    )
    
    legend_added = set()
    
    for idx, task in enumerate(tasks):
        row = idx // cols + 1
        col = idx % cols + 1
        
        task_data = data[data["task"] == task]
        if task_data.empty:
            continue
        
        stats = compute_stats(task_data, ["method", "step"])
        methods = ["Ours"] + [m for m in sorted(stats["method"].unique()) if m != "Ours"]
        
        for method in methods:
            method_stats = stats[stats["method"] == method].sort_values("step")
            if method_stats.empty:
                continue
            
            color = colors.get(method, "#666666")
            x = method_stats["step"]
            y_mean = method_stats["mean_reward"]
            y_std = method_stats["std_reward"].fillna(0)
            
            show_legend = method not in legend_added
            legend_added.add(method)
            
            # Shaded region
            fig.add_trace(
                go.Scatter(
                    x=pd.concat([x, x[::-1]]),
                    y=pd.concat([y_mean + y_std, (y_mean - y_std)[::-1]]),
                    fill="toself",
                    fillcolor=hex_to_rgba(color, 0.15),
                    line=dict(color="rgba(0,0,0,0)"),
                    showlegend=False,
                    hoverinfo="skip",
                    legendgroup=method,
                ),
                row=row, col=col
            )
            
            # Mean line
            line_style = dict(color=color, width=1.5)
            if method != "Ours":
                line_style["dash"] = "dash"
            
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=y_mean,
                    mode="lines",
                    name=method,
                    line=line_style,
                    showlegend=show_legend,
                    legendgroup=method,
                    hovertemplate=f"{method}: %{{y:.0f}}<extra></extra>",
                ),
                row=row, col=col
            )
    
    fig.update_layout(
        height=280 * rows,
        width=1000,
        title_text="<b>DM-Control Benchmark (2 Enc Layers): Ours vs Baselines</b>",
        title_x=0.5,
        showlegend=True,
        legend=dict(x=1.01, y=1, bgcolor="rgba(255,255,255,0.9)"),
        template="plotly_white",
    )
    
    return fig

In [14]:
grid_fig = create_grid_figure(all_data, our_tasks, COLORS, cols=3)
grid_fig.show()

## Normalized Aggregate Performance

Average performance across all tasks, normalized by max performance per task.

In [15]:
def compute_normalized_aggregate(
    data: pd.DataFrame,
    eval_step: int = 100_000,
    common_steps: list = None,
) -> pd.DataFrame:
    """Compute normalized aggregate performance across tasks.
    
    For each task, normalize rewards by max performance at eval_step.
    Interpolate to common steps, then average across tasks per method.
    
    Args:
        data: Combined DataFrame with [step, reward, method, task].
        eval_step: Step at which to compute max for normalization.
        common_steps: List of steps to interpolate to. If None, uses [0, 10k, ..., eval_step].
    
    Returns:
        DataFrame with [method, step, mean_normalized, std_normalized, n_tasks].
    """
    if common_steps is None:
        common_steps = list(range(0, eval_step + 1, 10_000))
    
    # First, compute max reward per task at eval_step (across all methods)
    task_max = {}
    for task in data["task"].unique():
        task_data = data[data["task"] == task]
        max_reward = 0
        for method in task_data["method"].unique():
            method_data = task_data[task_data["method"] == method]
            available_steps = method_data["step"].unique()
            valid_steps = available_steps[available_steps <= eval_step]
            if len(valid_steps) > 0:
                step = valid_steps.max()
                rewards_at_step = method_data[method_data["step"] == step]["reward"]
                max_reward = max(max_reward, rewards_at_step.mean())
        task_max[task] = max_reward if max_reward > 0 else 1
    
    print("Max rewards per task (for normalization):")
    for task, max_r in sorted(task_max.items()):
        print(f"  {task}: {max_r:.1f}")
    
    # Normalize all rewards
    data_normalized = data.copy()
    data_normalized["normalized_reward"] = data_normalized.apply(
        lambda r: r["reward"] / task_max[r["task"]], axis=1
    )
    
    # Interpolate each (method, task, seed) to common steps
    interpolated_rows = []
    for (method, task), group in data_normalized.groupby(["method", "task"]):
        # Average across seeds first, then interpolate
        seed_avg = group.groupby("step")["normalized_reward"].mean().reset_index()
        seed_avg = seed_avg.sort_values("step")
        
        # Interpolate to common steps
        for target_step in common_steps:
            if target_step < seed_avg["step"].min():
                # Extrapolate from first value (or use 0)
                val = seed_avg["normalized_reward"].iloc[0] if len(seed_avg) > 0 else 0
            elif target_step > seed_avg["step"].max():
                # Use last available value
                val = seed_avg["normalized_reward"].iloc[-1]
            else:
                # Linear interpolation
                val = np.interp(target_step, seed_avg["step"], seed_avg["normalized_reward"])
            
            interpolated_rows.append({
                "method": method,
                "task": task,
                "step": target_step,
                "normalized_reward": val,
            })
    
    interpolated_df = pd.DataFrame(interpolated_rows)
    
    # Aggregate across tasks
    aggregate = interpolated_df.groupby(["method", "step"]).agg(
        mean_normalized=("normalized_reward", "mean"),
        std_normalized=("normalized_reward", "std"),
        n_tasks=("normalized_reward", "count")
    ).reset_index()
    
    return aggregate

In [16]:
def plot_normalized_aggregate(
    aggregate_df: pd.DataFrame,
    colors: dict,
) -> go.Figure:
    """Plot normalized aggregate performance across tasks.
    
    Args:
        aggregate_df: DataFrame from compute_normalized_aggregate.
        colors: Dict mapping method names to hex colors.
    
    Returns:
        Plotly Figure.
    """
    fig = go.Figure()
    
    # Plot Ours first, then baselines
    methods = ["Ours"] + [m for m in sorted(aggregate_df["method"].unique()) if m != "Ours"]
    
    for method in methods:
        method_data = aggregate_df[aggregate_df["method"] == method].sort_values("step")
        if method_data.empty:
            continue
        
        color = colors.get(method, "#666666")
        x = method_data["step"].values
        y_mean = method_data["mean_normalized"].values
        y_std = np.nan_to_num(method_data["std_normalized"].values, nan=0)
        
        # Shaded std region - use numpy arrays for proper concatenation
        x_fill = np.concatenate([x, x[::-1]])
        y_fill = np.concatenate([y_mean + y_std, (y_mean - y_std)[::-1]])
        
        fig.add_trace(go.Scatter(
            x=x_fill,
            y=y_fill,
            fill="toself",
            fillcolor=hex_to_rgba(color, 0.2),
            line=dict(color="rgba(0,0,0,0)"),
            showlegend=False,
            hoverinfo="skip",
        ))
        
        # Mean line
        line_style = dict(color=color, width=2.5)
        if method != "Ours":
            line_style["dash"] = "dash"
            line_style["width"] = 2
        
        fig.add_trace(go.Scatter(
            x=x,
            y=y_mean,
            mode="lines",
            name=method,
            line=line_style,
            hovertemplate=f"{method}<br>Step: %{{x:,}}<br>Normalized: %{{y:.3f}}<extra></extra>",
        ))
    
    fig.update_layout(
        title=dict(text="<b>Normalized Aggregate Performance (Mean over Tasks)</b>", x=0.5),
        xaxis_title="Environment Steps",
        yaxis_title="Normalized Reward (fraction of max)",
        legend=dict(x=0.02, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
        hovermode="x unified",
        template="plotly_white",
        width=800,
        height=500,
        yaxis=dict(range=[0, 1.05]),
    )
    
    return fig

In [17]:
# Compute and plot normalized aggregate
aggregate_df = compute_normalized_aggregate(all_data, eval_step=MAX_STEP)
aggregate_fig = plot_normalized_aggregate(aggregate_df, COLORS)
aggregate_fig.show()

Max rewards per task (for normalization):
  acrobot-swingup: 480.9
  cartpole-swingup-sparse: 663.0
  cheetah-run: 824.7
  finger-spin: 988.8
  finger-turn-hard: 976.5
  hopper-hop: 591.8
  pendulum-swingup: 836.5
  quadruped-walk: 964.3
  reacher-hard: 921.9


In [18]:
# Debug: Check step alignment per method
print("Step counts and ranges per method:")
for method in all_data["method"].unique():
    method_data = all_data[all_data["method"] == method]
    steps = sorted(method_data["step"].unique())
    print(f"\n{method}:")
    print(f"  Unique steps: {len(steps)}")
    print(f"  Range: {steps[0]} - {steps[-1]}")
    print(f"  First 5 steps: {steps[:5]}")
    print(f"  Last 5 steps: {steps[-5:]}")

Step counts and ranges per method:

Ours:
  Unique steps: 10
  Range: 10000 - 100000
  First 5 steps: [10000, 20000, 30000, 40000, 50000]
  Last 5 steps: [60000, 70000, 80000, 90000, 100000]

TDMPC2:
  Unique steps: 2
  Range: 0 - 100000
  First 5 steps: [0, 100000]
  Last 5 steps: [0, 100000]

DreamerV3:
  Unique steps: 2
  Range: 0 - 100000
  First 5 steps: [0, 100000]
  Last 5 steps: [0, 100000]

SAC:
  Unique steps: 2
  Range: 0 - 100000
  First 5 steps: [0, 100000]
  Last 5 steps: [0, 100000]

EZ2:
  Unique steps: 72
  Range: 200 - 96000
  First 5 steps: [200, 300, 5700, 5800, 5900]
  Last 5 steps: [91000, 95700, 95800, 95900, 96000]

SimbaV2:
  Unique steps: 2
  Range: 0 - 100000
  First 5 steps: [0, 100000]
  Last 5 steps: [0, 100000]

BMPC:
  Unique steps: 2
  Range: 0 - 100000
  First 5 steps: [0, 100000]
  Last 5 steps: [0, 100000]


## Summary Table

In [19]:
def create_summary_table(data: pd.DataFrame, eval_step: int = None) -> pd.DataFrame:
    """Create summary table with final mean/std per method per task.
    
    Args:
        data: Combined DataFrame.
        eval_step: Step at which to evaluate. If None, uses max step per method.
    
    Returns:
        DataFrame with one row per task, columns for each method's mean/std.
    """
    results = []
    methods = sorted(data["method"].unique())
    
    for task in sorted(data["task"].unique()):
        task_data = data[data["task"] == task]
        row = {"task": task}
        
        for method in methods:
            method_data = task_data[task_data["method"] == method]
            if method_data.empty:
                row[f"{method}_mean"] = np.nan
                row[f"{method}_std"] = np.nan
                continue
            
            # Get data at eval step or max available
            if eval_step is not None:
                # Find closest step <= eval_step
                available_steps = method_data["step"].unique()
                valid_steps = available_steps[available_steps <= eval_step]
                if len(valid_steps) == 0:
                    step = available_steps.max()
                else:
                    step = valid_steps.max()
            else:
                step = method_data["step"].max()
            
            final_data = method_data[method_data["step"] == step]
            row[f"{method}_mean"] = final_data["reward"].mean()
            row[f"{method}_std"] = final_data["reward"].std()
        
        results.append(row)
    
    return pd.DataFrame(results)

In [20]:
# Summary at 100k steps (this sweep's max)
summary_100k = create_summary_table(all_data, eval_step=100_000)
print("Summary at 100k steps (or max available):")
summary_100k.round(1)

Summary at 100k steps (or max available):


,task,BMPC_mean,BMPC_std,DreamerV3_mean,DreamerV3_std,EZ2_mean,EZ2_std,Ours_mean,Ours_std,SAC_mean,SAC_std,SimbaV2_mean,SimbaV2_std,TDMPC2_mean,TDMPC2_std
0,acrobot-swingup,203.8,55.2,101.3,22.3,307.6,60.7,480.9,87.4,7.7,11.3,102.7,56.9,179.1,85.3
1,cartpole-swingup-sparse,269.9,329.9,315.3,191.3,663.0,159.2,41.3,25.9,0.4,0.6,557.7,318.9,0.3,0.6
2,cheetah-run,506.0,81.2,335.7,63.0,659.2,22.5,824.7,64.6,310.1,65.3,601.5,87.7,519.1,83.8
3,finger-spin,943.5,28.8,328.3,58.6,753.5,62.6,988.8,1.4,762.4,16.5,744.9,170.1,947.5,34.2
4,finger-turn-hard,215.2,149.9,189.8,93.0,239.7,77.0,976.5,12.0,61.7,45.6,533.2,192.1,483.6,97.0
5,hopper-hop,58.5,63.1,9.7,16.7,152.1,106.3,591.8,44.9,0.0,0.0,176.4,111.6,13.6,12.8
6,pendulum-swingup,794.7,26.4,699.3,209.9,827.7,NaN,836.5,28.4,96.0,94.1,826.0,25.2,724.0,146.9
7,quadruped-walk,342.6,84.2,45.9,27.9,873.7,16.9,964.3,0.0,55.2,33.7,906.1,42.9,486.8,311.9
8,reacher-hard,648.7,282.0,813.3,47.0,759.6,131.9,921.9,67.7,161.3,187.7,890.4,80.9,414.2,137.5


In [21]:
def format_summary_pretty(df: pd.DataFrame) -> pd.DataFrame:
    """Format summary with 'mean ± std' columns."""
    methods = sorted(set(col.replace("_mean", "") for col in df.columns if col.endswith("_mean")))
    
    formatted = df[["task"]].copy()
    for method in methods:
        mean_col = f"{method}_mean"
        std_col = f"{method}_std"
        if mean_col in df.columns:
            formatted[method] = df.apply(
                lambda r: f"{r[mean_col]:.0f} ± {r[std_col]:.0f}" 
                if pd.notna(r[mean_col]) else "—",
                axis=1
            )
    return formatted

pretty_summary = format_summary_pretty(summary_100k)
pretty_summary

,task,BMPC,DreamerV3,EZ2,Ours,SAC,SimbaV2,TDMPC2
0,acrobot-swingup,204 ± 55,101 ± 22,308 ± 61,481 ± 87,8 ± 11,103 ± 57,179 ± 85
1,cartpole-swingup-sparse,270 ± 330,315 ± 191,663 ± 159,41 ± 26,0 ± 1,558 ± 319,0 ± 1
2,cheetah-run,506 ± 81,336 ± 63,659 ± 23,825 ± 65,310 ± 65,602 ± 88,519 ± 84
3,finger-spin,944 ± 29,328 ± 59,753 ± 63,989 ± 1,762 ± 17,745 ± 170,948 ± 34
4,finger-turn-hard,215 ± 150,190 ± 93,240 ± 77,976 ± 12,62 ± 46,533 ± 192,484 ± 97
5,hopper-hop,58 ± 63,10 ± 17,152 ± 106,592 ± 45,0 ± 0,176 ± 112,14 ± 13
6,pendulum-swingup,795 ± 26,699 ± 210,828 ± nan,836 ± 28,96 ± 94,826 ± 25,724 ± 147
7,quadruped-walk,343 ± 84,46 ± 28,874 ± 17,964 ± 0,55 ± 34,906 ± 43,487 ± 312
8,reacher-hard,649 ± 282,813 ± 47,760 ± 132,922 ± 68,161 ± 188,890 ± 81,414 ± 137


## Save Results

In [22]:
# Create output directory
output_dir = paths.notebook_results_dir("11_dmcontrol_2enc_benchmark")
print(f"Output directory: {output_dir}")

# Save summary tables
summary_100k.to_csv(output_dir / "summary_100k.csv", index=False)
pretty_summary.to_csv(output_dir / "summary_100k_pretty.csv", index=False)

# Save grid figure
grid_fig.write_html(output_dir / "grid_comparison.html")

# Save individual task figures
for task, fig in figures.items():
    safe_name = task.replace("-", "_")
    fig.write_html(output_dir / f"{safe_name}.html")

print(f"\nSaved:")
print(f"  - summary_100k.csv")
print(f"  - summary_100k_pretty.csv")
print(f"  - grid_comparison.html")
print(f"  - {len(figures)} individual task plots")

Output directory: /gpfs/work4/0/prjs0951/Thomas/Thesis/RL_weather/tdmpc2-with-return-based-auxiliary-tasks.worktrees/main/analysis/results/11_dmcontrol_2enc_benchmark

Saved:
  - summary_100k.csv
  - summary_100k_pretty.csv
  - grid_comparison.html
  - 9 individual task plots
